In [ ]:
%%javascript
(function(){
  function clickConnect(){
    const host = document.querySelector('colab-connect-button');
    const btn = host && host.shadowRoot ? host.shadowRoot.querySelector('#connect') : null;
    (btn || document.querySelector('#connect'))?.click();
    console.log('⏳ Mantendo sessão ativa...');
  }
  setInterval(clickConnect, 60000);
})();

In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y python3.10 python3.10-distutils
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10
!python3 --version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf /content/stable-diffusion-webui
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/stable-diffusion-webui

In [ ]:
!mkdir -p /content/stable-diffusion-webui/models/Lora
!ln -s /content/drive/MyDrive/SD_LoRAs /content/stable-diffusion-webui/models/Lora

In [ ]:
%cd /content/stable-diffusion-webui
!COMMANDLINE_ARGS="--share --disable-safe-unpickle --skip-torch-cuda-test --precision full --no-half" \
  REQS_FILE="requirements.txt" \
  python launch.py